# PyTorch Assignment: Convolutional Neural Network (CNN)

**[Duke Community Standard](http://integrity.duke.edu/standard.html): By typing your name below, you are certifying that you have adhered to the Duke Community Standard in completing this assignment.**

Name: Abhinav Tembulkar

### Convolutional Neural Network

Adapt the CNN example for MNIST digit classfication from Notebook 3A. 
Feel free to play around with the model architecture and see how the training time/performance changes, but to begin, try the following:

Image ->  
convolution (32 3x3 filters) -> nonlinearity (ReLU) ->  
convolution (32 3x3 filters) -> nonlinearity (ReLU) -> (2x2 max pool) ->  
convolution (64 3x3 filters) -> nonlinearity (ReLU) ->  
convolution (64 3x3 filters) -> nonlinearity (ReLU) -> (2x2 max pool) -> flatten ->
fully connected (256 hidden units) -> nonlinearity (ReLU) ->  
fully connected (10 hidden units) -> softmax 

Note: The CNN model might take a while to train. Depending on your machine, you might expect this to take up to half an hour. If you see your validation performance start to plateau, you can kill the training.

In [1]:
import torch
import torch.functional as F
import torchvision
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
from tqdm.notebook import tqdm

train = torchvision.datasets.MNIST(root='./datasets/', transform=torchvision.transforms.ToTensor(), download=True, train=True)
test = torchvision.datasets.MNIST(root='./datasets/', transform=torchvision.transforms.ToTensor(), download=True, train=False)

train_loader = torch.utils.data.DataLoader(train, batch_size=100, shuffle=True)
test_loader = torch.utils.data.DataLoader(test, batch_size=100, shuffle=False)
trainal, testal = next(iter(train_loader)) 
print(trainal.shape, testal.shape)
class CNN(nn.Module):
	def __init__(self):
		super().__init__()

		self.layers = nn.Sequential(
			nn.Conv2d(1, 32, kernel_size=3),
			nn.ReLU(),
			nn.Conv2d(32, 32, kernel_size=3),
			nn.ReLU(),
			nn.MaxPool2d(kernel_size=2),
			nn.Conv2d(32, 64, kernel_size=3),
			nn.ReLU(),
			nn.Conv2d(64, 64, kernel_size=3),
			nn.ReLU(),
			nn.MaxPool2d(kernel_size=2),
			nn.Flatten(),
			nn.Linear(1024, 256),
			nn.ReLU(),
			nn.Linear(256, 10)
			)

	def forward(self, x):
		out = self.layers(x)
		return out
model = CNN()
optimizer = torch.optim.Adam(model.parameters(), lr=.01)
loss_fn = torch.nn.CrossEntropyLoss()
loss_arr = []

for epoch in range (10):
	print(f'EPOCH: {epoch}')
	for train, test in tqdm(train_loader):
		optimizer.zero_grad()
		x = train
		y = model(x)
		loss = loss_fn(y, test)
		loss_arr.append(loss)
		loss.backward()
		optimizer.step()


In [34]:
torch.save(model.state_dict(), 'model.pt')	